<hr>
<b style='color: #a11d21;font-size:36px' id="data_understanding"> Welcome to Middle Earth: Facebook Scrapping</b>

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Choose your character: Import your packages</b>

In [1]:
import requests
import re
import json
import time
import logging
import pandas
from collections import OrderedDict
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
import re
import json
import time
import logging
from collections import OrderedDict
from tqdm import tqdm_notebook as tqdm
from random import seed
from random import randint
from datetime import datetime
from webdriver_manager.chrome import ChromeDriverManager
from time import time       # Other imports
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import csv
from selenium import webdriver
#Importing packages
from bs4 import BeautifulSoup
from random import seed
from random import randint
from datetime import datetime

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Choose your weapons: Secondary Functions </b>

In [2]:
def get_bs(session, url):
    """Makes a GET requests using the given Session object
    and returns a BeautifulSoup object.
    """
    r = None
    while True:
        r = session.get(url)
        if r.ok:
            break
    return BeautifulSoup(r.text, 'lxml')

def make_login(session, base_url, credentials):
    """Returns a Session object logged in with credentials.
    """
    login_form_url = '/login/device-based/regular/login/?refsrc=https%3A'\
        '%2F%2Fmobile.facebook.com%2Flogin%2Fdevice-based%2Fedit-user%2F&lwv=100'
 
    params = {'email':credentials['email'], 'pass':credentials['pass']}
 
    while True:
        time.sleep(3)
        logged_request = session.post(base_url+login_form_url, data=params)
        
        if logged_request.ok:
            logging.info('[*] Logged in.')
            break
            
def crawl_profile(session, base_url, profile_url, post_limit):
    """Goes to profile URL, crawls it and extracts posts URLs.
    """
    profile_bs = get_bs(session, profile_url)
    n_scraped_posts = 0
    scraped_posts = list()
    posts_id = None
 
    while n_scraped_posts < post_limit:
        print("n_scraped_posts")
        try:
            posts_id = 'recent'
            posts = profile_bs.find('div', id=posts_id).div.div.contents
            print(posts)
        except Exception:
            posts_id = 'structured_composer_async_container'
            posts = profile_bs.find('div', id=posts_id).div.div.contents
            print(posts)
        posts_urls = [a['href'] for a in profile_bs.find_all('a', text='Full Story')] 
        
        print(len(posts_urls))
        for post_url in posts_urls:
            # print(post_url)
            try:
                post_data = scrape_post(session, base_url, post_url)
                scraped_posts.append(post_data)
            except Exception as e:
                logging.info('Error: {}'.format(e))
            n_scraped_posts += 1
            if posts_completed(scraped_posts, post_limit):
                break
        
        #show_more_posts_url = None
        #if not posts_completed(scraped_posts, post_limit):
        #    show_more_posts_url = profile_bs.find('div', id=posts_id).next_sibling.a['href']
        #    profile_bs = get_bs(session, base_url+show_more_posts_url)
        #    time.sleep(3)
        #else:
        #    break
    return scraped_posts
 
def posts_completed(scraped_posts, limit):
    """Returns true if the amount of posts scraped from
    profile has reached its limit.
    """
    if len(scraped_posts) == limit:
        return True
    else:
        return False
 
 
def scrape_post(session, base_url, post_url):
    """Goes to post URL and extracts post data.
    """
    post_data = OrderedDict()
 
    post_bs = get_bs(session, base_url+post_url)
    time.sleep(2)
 
    # Here we populate the OrderedDict object
    post_data['url'] = post_url
 
    try:
        post_text_element = post_bs.find('div', id='u_0_0').div
        string_groups = [p.strings for p in post_text_element.find_all('p')]
        strings = [repr(string) for group in string_groups for string in group]
        post_data['text'] = strings
    except Exception:
        post_data['text'] = []
    
    try:
        post_data['media_url'] = post_bs.find('div', id='u_0_0').find('a')['href']
    except Exception:
        post_data['media_url'] = ''
    
 
    try:
        post_data['comments'] = extract_comments(session, base_url, post_bs, post_url)
    except Exception:
        post_data['comments'] = []
    
    return dict(post_data)            

def extract_comments(session, base_url, post_bs, post_url):
    """Extracts all coments from post
    """
    comments = list()
    show_more_url = post_bs.find('a', href=re.compile('/story\.php\?story'))['href']
    first_comment_page = True
 
    logging.info('Scraping comments from {}'.format(post_url))
    while True:
 
        logging.info('[!] Scraping comments.')
        time.sleep(2)
        if first_comment_page:
            first_comment_page = False
        else:
            post_bs = get_bs(session, base_url+show_more_url)
            time.sleep(2)
        
        try:
            comments_elements = post_bs.find('div', id=re.compile('composer')).next_sibling\
                .find_all('div', id=re.compile('^\d+'))
        except Exception:
            pass
 
        if len(comments_elements) != 0:
            logging.info('[!] There are comments.')
        else:
            break
        
        for comment in comments_elements:
            comment_data = OrderedDict()
            comment_data['text'] = list()
            try:
                comment_strings = comment.find('h3').next_sibling.strings
                for string in comment_strings:
                    comment_data['text'].append(string)
            except Exception:
                pass
            
            try:
                media = comment.find('h3').next_sibling.next_sibling.children
                if media is not None:
                    for element in media:
                        comment_data['media_url'] = element['src']
                else:
                    comment_data['media_url'] = ''
            except Exception:
                pass
            
            comment_data['profile_name'] = comment.find('h3').a.string
            comment_data['profile_url'] = comment.find('h3').a['href'].split('?')[0]
            comments.append(dict(comment_data))
        
        show_more_url = post_bs.find('a', href=re.compile('/story\.php\?story'))
        if 'View more' in show_more_url.text:
            logging.info('[!] More comments.')
            show_more_url = show_more_url['href']
        else:
            break
    
    return comments

def json_to_obj(filename):
    """Extracts dta from JSON file and saves it on Python object
    """
    obj = None
    with open(filename) as json_file:
        obj = json.loads(json_file.read())
    return obj

def save_data(data):
    """Converts data to JSON.
    """
    with open('profile_posts_data.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)
        
def get_bs(session, url):
    """Makes a GET requests using the given Session object
    and returns a BeautifulSoup object.
    """
    r = None
    while True:
        r = session.get(url)
        if r.ok:
            break
    return BeautifulSoup(r.text, 'lxml')

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Start the Adventure: Open WebDriver  </b>

In [3]:
# Choose your destination file
driver = webdriver.Chrome('C:/Users/GSantos/Documents/GABRIEL RAVI/Web Scrapping/chromedriver_win32_2/chromedriver.exe')

In [4]:
#Check the field
driver.get('https://mobile.facebook.com/story.php?story_fbid=4082772431741189&id=122118287806643&anchor_composer=false')


<b style='color: #30475e;font-size:12px' id="data_understanding"> You will be directed to the log in face in facebook  </b>
<hr>

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Meet Gollum: Log in Facebook </b>

Open the driver and log in to your facebook account

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Find the resources: Get the Updated List </b>

In [22]:
# Select your list
base_url = 'https://mobile.facebook.com'
fb = pd.read_excel('Lista_19.23.21_Ravi.xlsx')

fb['post_author_url'] = [re.sub(r'/$', "", x) for x in fb['post_author_url']]
fb['posts'] = [re.sub(r'https://www.facebook.com',base_url, x) for x in fb['post_author_url']] 
fb['posts'] = fb['posts'] + '/posts/?ref=page_internal&mt_nav=0'

fb

,Setor,Categoria,Empresa,post_author_url,tag,posts
0,Academia,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,F,https://mobile.facebook.com/uegas/posts/?ref=p...
1,Academia,Academia,ESEI Maria Ulrich,https://www.facebook.com/esei.maria.ulrich,F,https://mobile.facebook.com/esei.maria.ulrich/...
2,Academia,Academia,Instituto Piaget,https://www.facebook.com/institutopiaget,F,https://mobile.facebook.com/institutopiaget/po...
3,Academia,Academia,Instituto Superior Tecnico,https://www.facebook.com/tecnico.ulisboa.pt,F,https://mobile.facebook.com/tecnico.ulisboa.pt...
4,Academia,Academia,ISCTE,https://www.facebook.com/ISCTEIUL,F,https://mobile.facebook.com/ISCTEIUL/posts/?re...
...,...,...,...,...,...,...
195,Utilities,Energia,ALFA Energia,https://www.facebook.com/EnergiaALFA,G,https://mobile.facebook.com/EnergiaALFA/posts/...
196,Utilities,Energia,ENAT Energias,https://www.facebook.com/enatenergias,G,https://mobile.facebook.com/enatenergias/posts...
197,Utilities,Energia,Energia Simples,https://www.facebook.com/energiasimples,G,https://mobile.facebook.com/energiasimples/pos...
198,Utilities,Energia,Luzboa,https://www.facebook.com/luzboa.energia,G,https://mobile.facebook.com/luzboa.energia/pos...


<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Explore Middle Earth: Scroll Down function </b>

In [23]:
def scroll(driver): #, timeout,min_posts
    min_posts=400
    #scroll_pause_time = timeout
    #n_scraped_posts_antes = -1
    seed(2019)
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    print(last_height)
    name_pattern = re.compile(r'2018', flags = re.M)

    while True: #n_scraped_posts < min_posts:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(randint(0, 10))

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        page_source = driver.page_source
        profile_bs = BeautifulSoup(page_source, 'lxml')
        n_scraped_posts = len(profile_bs.findAll('a',  text = re.compile('comen|Comen')))
        

        data_fim = driver.find_elements_by_tag_name('abbr')        
        data_fim= data_fim[len(data_fim)-1]

        if len(name_pattern.findall(data_fim.text))>0:
            break

        if n_scraped_posts >= min_posts:
            # If number of colected posts are the same or higher it will exit the function
            break
        
        #if n_scraped_posts_antes == n_scraped_posts:
            #if new_height == last_height:
                # If heights are the same it will exit the function
                #break
        last_height = new_height    
        #n_scraped_posts_antes = n_scraped_posts
            

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Fight against Orcs: Activate the two main Functions </b>

In [7]:
def info_post():
    
    global partilhas
    global texto_post
    global data_post
    global contagem_reacoes
    #global autor_post_id
    global tipo_post_video
    global tipo_post_foto
    global tipo_post_evento
    global tipo_post_link
    global total_replies
    
    data_post = []
    try:
        data_post = driver.find_element_by_tag_name('abbr').text
    except:
        data_post = ["NaN"]
    ####################
    texto_post = []
    try:
        for texto_post_ in driver.find_elements_by_tag_name('p'):
                texto_post.append(texto_post_.text)        
        texto_post  = [''.join(texto_post)]    
    except:
        pass  
    if texto_post == ['']:
        try:
            texto_post0 = driver.find_elements_by_xpath("//*[@id='MPhotoUpperContent']/div/div/div[1]/div/div/div[1]/div")
            for texto_post_ in texto_post0:
                texto_post.append(texto_post_.text)
            texto_post  = [''.join(texto_post)] 
        except:
            pass
    if texto_post == ['']:
        texto_post = ["NaN"]   
##################################
    page_source = driver.page_source
    profile_bs = BeautifulSoup(page_source, 'lxml')

    partilhas = profile_bs.findAll('span', {'data-sigil': 'feed-ufi-sharers'})

    partilhas= re.compile(r'[0-9]').findall(str(partilhas))
    partilhas = ''.join(partilhas)
    
    #name_pattern = re.compile(r'href=".+?"', flags = re.M)

    #autor_post_id = profile_bs.find('a', href=re.compile('\/.refid|story|/\?_'))
    #autor_post_id = [re.sub(r'"|href=|\?.*|\/','', x) for x in name_pattern.findall(str(autor_post_id))]
    #return autor_post_id      
    
    # Video
    name_pattern = re.compile(r'playInlineVideo', flags = re.M)
    tipo_post_video = len(name_pattern.findall(str(profile_bs.find('div', {'class': 'story_body_container'}))))/2
    # Foto
    name_pattern = re.compile(r'href.+?/photos/', flags = re.M)
    tipo_post_foto = len(name_pattern.findall(str(profile_bs.find('div', {'class': 'story_body_container'}))))
    # Eventos
    name_pattern = re.compile(r'href.+?/events/', flags = re.M)
    tipo_post_evento = len(name_pattern.findall(str(profile_bs.find('div', {'class': 'story_body_container'}))))
    # Link
    name_pattern = re.compile(r'href.+?lm.facebook', flags = re.M)
    tipo_post_link = len(name_pattern.findall(str(profile_bs.find('div', {'class': 'story_body_container'}))))
    
    try:
        reacoes = base_url + profile_bs.find('a', href=re.compile('/ufi/reaction'))['href']
    except:
        return
    
    total_replies = len(driver.find_elements_by_xpath("//div[@class='_2b04']//div[@class='_2b06']")) - len(driver.find_elements_by_css_selector("._2a_i._2a_l"))

    driver.get(reacoes)
    
    
    page_source = driver.page_source
    profile_bs = BeautifulSoup(page_source, 'lxml')

    contagem_reacoes = profile_bs.findAll('span', {'class': '_10tn'})

    name_pattern = re.compile(r'Type":.{4}|aria-label=.+?pessoa.', flags = re.M)
    # Find all occurrences of the pattern
    contagem_reacoes = name_pattern.findall(str(contagem_reacoes))

    contagem_reacoes = [re.sub(r'"|\xa0|\'>|Type":|}|\\|aria-label="| pessoa..?','', x) for x in contagem_reacoes]


In [8]:
#If prints, everything is OK!

info_post()
tipo_post_video
texto_post

['O Som do Metal, nomeado para os Óscares, está agora disponível no Videoclube da Tv da Vodafone.']

In [9]:
#Check the field
driver.get('https://mobile.facebook.com/story.php?story_fbid=4082772431741189&id=122118287806643&anchor_composer=false')

In [10]:
# It need to be logged in a pub
def scrap_comments():
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    dms = driver.find_elements_by_css_selector('._2b1h.async_elem')
    for i in dms:
        i.click()
    
    sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    
    try:
        dms1 = driver.find_elements_by_css_selector('.async_elem')
        for i in dms1:
            i.click()
    except:
        sleep(1)
    sleep(2)
   
    global comments2
    global comment_id2
    global autor2
    global like_comentario2
    global hora2
    global autor_id2
    global n_comments2
    global n_respostas2
    global comment_url2
    global qt_replies2
    
    all_comments = driver.find_elements_by_xpath("//div[@data-commentid]")
    autor_comentario = driver.find_elements_by_class_name('_2b05')
    likes_comentario = driver.find_elements_by_class_name('_14va')
    hora_comentario = driver.find_elements_by_xpath("//abbr[@classas]")
    
    comments = []
    n_resp_ = []
    autor = []
    like_comentario = []
    hora = []
    comment_url = []
    n_c = []
    n_resp_ = []
    qt_replies = []
    
    for comment in all_comments:
        comments.append(comment.text)
    
    for autor0 in autor_comentario:
        autor.append(autor0.text)

    for like_comentario0 in likes_comentario:
        like_comentario.append(like_comentario0.text)

    for horario in hora_comentario:
        hora.append(horario.text)
###################################3
    page_source = driver.page_source
    profile_bs = BeautifulSoup(page_source, 'lxml')

    name_pattern = re.compile(r'id=".+?"')

    comment_id = profile_bs.findAll('div', {'data-sigil': 'comment-body'})
    comment_id = name_pattern.findall(str(comment_id))

    comment_id = [re.sub(r'"|id=','', x) for x in comment_id]

    name_pattern = re.compile(r'ref=".+?"')
    #name_pattern = re.compile(r'(?:id=.+?&|<a href="\/.+?\?fnf|.+?\?rc)')
    
    autor_id = profile_bs.findAll('div', {'class': '_2b05'})
    autor_id = name_pattern.findall(str(autor_id))

    autor_id = [re.sub(r'"|ref=|profile.php\?id=|\&.*','', x) for x in autor_id]
    autor_id = [re.sub(r'\?fnf|\?rc=p|\/','', x) for x in autor_id]
    
    if comments==[]:
        n_c = ['']
    else:
        name_pattern = re.compile(r'_2a_j _874d')
        n_comments = profile_bs.findAll('div', {'class': '_2a_j'})
        for rep in n_comments:
            aux = name_pattern.findall(str(rep))
            if aux == []:
                aux = ['']
            n_c = np.concatenate((n_c,aux))
        n_c = [re.sub(r'_2a_j _874d','1', x) for x in n_c]
            
    
###### Flag Reply    
    
######################    
    for comment_url0 in profile_bs.findAll('a',  href = re.compile('comment')):
        comment_url.append(base_url + comment_url0['href'])
    
    #n_resp += np.concatenate((n_resp, list(np.tile('',len(comments)))))
#######################    
    global replies2
    
    replies1 = []
    if comments==[]:
        replies1 = ['']
    else:
                name_pattern = re.compile(r'_2a_j _874d')
               # _333v _45kb
                n_comments = profile_bs.findAll('div', {'class': '_2a_j'})
                for rep in n_comments:
                            aux = len(name_pattern.findall(str(rep)))
                            if aux == []:
                                aux = ['']    

                            replies1.append(aux)
                #            replies1.extend(["0"] * aux)

    
######################        
    comment_url2 += comment_url
#    n_respostas2 += n_resp_
    n_comments2 += n_c
    comments2 += comments
    comment_id2 += comment_id
    autor2 += autor
    like_comentario2 += like_comentario
    hora2 += hora
    autor_id2 += autor_id
    qt_replies2 += qt_replies
    replies2 += replies1

In [11]:
comments2 = [] 
autor2 = [] 
like_comentario2 = [] 
hora2 = [] 
comment_id2 = []
autor_id2 = []
comment_url2 = []
n_comments2 = []
n_respostas2 = []
qt_replies2 = []
replies2 = []
scrap_comments()

#If prints, everything is OK!
comments2

['talvez dos piores filmes que já vi, sem essência nenhuma e até ao final nada se passa !',
 'I really liked this page..',
 'Me too',
 'Me too again',
 'quase adormeci',
 'Eu adormeci',
 'Sónia Nunes tens a certeza que não é? 😃😃😃😃',
 'Carlos Alberto. Tu és o meu primo filho do meu tio Gil Moreira. O grande ciclista que fez a primeira etapa de França Portugal. Deu tanta roupa há minha mãe Julia Moreira sua irmã. O seu primeiro carro que teve era um Citroën 2 Cavalos. Um grande abraço do vosso primo Amílcar Henriques Vieira (o Mica)',
 'Bom filme',
 'Amém']

In [12]:
replies2

[0, 0, 1, 1, 0, 1, 0, 0, 0, 0]

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Meet Gandalf The White: Create a Ordered Database  </b>

In [13]:
import time

In [33]:
start_time = time.time()

post_data = OrderedDict()

post_data['Categoria'] = []
post_data['Empresa'] = []
post_data['post_author_url'] = []
post_data['n'] = []
post_data['Empresa'] = []
post_data['post_author_url'] = []
post_data['n'] = []
post_data['URL_Post'] = []
post_data['Texto_Post'] = []
post_data['Data_Post'] = []
post_data['Partilhas'] = []
post_data['Tipo_post_video'] = []
post_data['Tipo_post_foto'] = []
post_data['Tipo_post_evento'] = []
post_data['Tipo_post_link'] = []
post_data['all'] = []
post_data['1'] = []
post_data['2'] = []
post_data['3'] = []
post_data['4'] = []
post_data['5'] = []
post_data['6'] = []
post_data['7'] = []
post_data['8'] = []
post_data['Likes_Comentario'] = []
post_data['Autor'] = []
post_data['Autor_id'] = []
post_data['Comentario']  = []
post_data['Comentario_id'] = []
post_data['Data_Comentario'] = []
post_data['N_respostas'] = []
post_data['Tempo_recolha'] = []
post_data['Flag_Reply'] = []
post_data['Replies'] = []
#post_data['Autor_Post_ID'] = []

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Fight against Sauron: Function that Scraps  </b>

<hr>
<b style='color: #a11d21;font-size:15px' id="data_understanding"> Go wisely: Split your list into chunks  </b>

In [25]:
# Get to know your dimensions
fb.shape

#Divide chunks with size 5 and the control it from here
print("Eu tenho")
print(len(fb['posts']))
print("links")

Eu tenho
200
links



<b style='color: #a11d21;font-size:12px' id="data_understanding"> Media: 0 to  70 </b>

<b style='color: #a11d21;font-size:12px' id="data_understanding"> Turismo: 71 to 101 </b>

In [ ]:
#Media
fb['posts'][0:199] 

#Chunks
fb['posts'][0:4] #OK
fb['posts'][5:12] 

#### Save your files with FB_Category_x_y

<hr>
<b style='color: #a11d21;font-size:15px' id="data_understanding"> Protect Gondor: Active the function  </b>

In [26]:
from tqdm.notebook import tqdm

In [ ]:
if __name__ == "__main__":
    
    size_ = 0
    with tqdm(total=len(fb['posts'][5:12])) as pbar:
        for company, lista_posts in enumerate(fb['posts'][5:12], start=0):

            pbar.set_description(f'Tamanho da base: {size_} obs')
            pbar.update(1)
            
            driver.get(lista_posts) #fb['posts'][4])

            scroll(driver)

            page_source = driver.page_source
            profile_bs = BeautifulSoup(page_source, 'lxml')

            posts_url = []

            for posts in profile_bs.findAll('a',  text = re.compile('comen|Comen')):
                posts_url.append(base_url + posts['href'])
            
            posts_url = posts_url[:(len(posts_url)-1)]
            posts_url = [x for x in posts_url if re.findall(r'^(?!.*ufi.reaction).*$', x, re.M)]
            posts_url = [x for x in posts_url if re.findall(r'^(?!.*lm.facebook).*$', x, re.M)]
            #print(company,fb['Empresa'][company],"com",len(posts_url)," posts a serem coletados")
            company_ = fb["Empresa"][company]

            with tqdm(total=len(posts_url)) as pbar2:
                for n, i in enumerate(posts_url, start=1):
                    pbar2.update(1)
                    pbar2.set_description(f'Posts da {company_}')
                    driver.get(i)

                    info_post()

                    driver.get(i)
                    
                    dms = driver.find_elements_by_css_selector('._2b1h.async_elem')
                    for i in dms:
                        i.click()
                    
                    comments2 = []
                    autor2 = []
                    like_comentario2 = []
                    hora2 = []
                    comment_id2 = []
                    autor_id2 = []
                    comment_url2 = []
                    n_comments2 = []                
                    comment_url2 = []
                    replies2 = []
                    #n_respostas2 = n_resp = []
                    
                    scrap_comments()
                    while len(driver.find_elements_by_xpath("//*[contains(@id,'see_next')]")) == 1:

                        page_source = driver.page_source
                        profile_bs = BeautifulSoup(page_source, 'lxml')

                        comment_url_aux = []

                        for posts in profile_bs.findAll('a', href=re.compile('/story')):
                            comment_url_aux.append(base_url + posts['href'])

                        if comment_url_aux == []:
                            break

                        comment_url_aux = comment_url_aux[len(comment_url_aux)-1]

                        driver.get(comment_url_aux)
                        dms = driver.find_elements_by_css_selector('._2b1h.async_elem')
                        for i in dms:
                            i.click()
                            
                        scrap_comments()

                    if len(comments2) == 0:
                        comments2, comment_id2, like_comentario2, autor2, autor_id2, hora2 = ["NaN"], ["NaN"], ["NaN"], ["NaN"], ["NaN"], ["NaN"]

                    #n_resp = np.tile('',len(comments2))
                    comment_comment = comment_url2
                    if len(comment_url2) > 0:
                        for comment_count, aux in enumerate(comment_comment,start=0):
                            comment_url2 = []
                            driver.get(comment_comment[comment_count])
                            
                            dms = driver.find_elements_by_css_selector('._2b1h.async_elem')
                            for i in dms:
                                i.click()
                            scrap_comments()
                        
                    post_data['Tempo_recolha'] = np.concatenate((post_data['Tempo_recolha'], np.tile(datetime.now(), len(comments2))))
                    post_data['Categoria'] = np.concatenate((post_data['Categoria'], np.tile(fb['Categoria'][company], len(comments2))))
                    post_data['Empresa'] = np.concatenate((post_data['Empresa'], np.tile(fb['Empresa'][company], len(comments2))))
                    post_data['post_author_url'] = np.concatenate((post_data['post_author_url'], np.tile(fb['post_author_url'][company], len(comments2))))
                    post_data['n'] = np.concatenate((post_data['n'], np.tile(n, len(comments2))))
                    post_data['URL_Post'] = np.concatenate((post_data['URL_Post'], np.tile(i, len(comments2))))
                    post_data['Texto_Post'] = np.concatenate((post_data['Texto_Post'], np.tile(texto_post, len(comments2))))
                    post_data['Data_Post'] = np.concatenate((post_data['Data_Post'], np.tile(data_post, len(comments2))))
                    post_data['Partilhas'] = np.concatenate((post_data['Partilhas'], np.tile(partilhas, len(comments2))))
                    #post_data['Autor_Post_ID'] = np.concatenate((post_data['Autor_Post_ID'], np.tile(autor_post_id, len(comments2))))
                    post_data['Tipo_post_video'] = np.concatenate((post_data['Tipo_post_video'], np.tile(tipo_post_video, len(comments2))))
                    post_data['Tipo_post_foto'] = np.concatenate((post_data['Tipo_post_foto'], np.tile(tipo_post_foto, len(comments2))))
                    post_data['Tipo_post_evento'] = np.concatenate((post_data['Tipo_post_evento'], np.tile(tipo_post_evento, len(comments2))))
                    post_data['Tipo_post_link'] = np.concatenate((post_data['Tipo_post_link'], np.tile(tipo_post_link, len(comments2))))

                    size_ = len(post_data['URL_Post'])

                    tipos_reacoes = list(contagem_reacoes[i] for i in range(0,len(contagem_reacoes),2))
                    valores_reacoes = list(contagem_reacoes[i] for i in range(1,len(contagem_reacoes),2))

                    for p in ['all', '1', '2', '3','4','5','6','7', '8']:
                        if p not in tipos_reacoes:
                            post_data[p] = np.concatenate((post_data[p], np.tile(0, max(1,len(comments2)))))
                        else:
                            post_data[p] = np.concatenate((post_data[p], np.tile(valores_reacoes[tipos_reacoes.index(p)],max(1,len(comments2)))))

                    post_data['Likes_Comentario'] = post_data['Likes_Comentario'] + like_comentario2
                    post_data['Autor'] = post_data['Autor'] + autor2
                    post_data['Autor_id'] = post_data['Autor_id'] + autor_id2
                    post_data['Comentario'] = post_data['Comentario'] + comments2
                    post_data['Comentario_id'] = post_data['Comentario_id'] + comment_id2
                    post_data['Data_Comentario'] = post_data['Data_Comentario'] + hora2
                    post_data['Flag_Reply'] = post_data['Flag_Reply'] + n_comments2
                    post_data['Replies'] = post_data['Replies'] + replies2
#                   post_data['N_respostas2'] = post_data['N_respostas2'] + n_respostas2

    print("\n--- %s minutes ---" % ((time.time() - start_time)/60))

1787



1392



1355


<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Gondor wins but Sauron has another army: Concatenate Databse  </b>

In [28]:
post_data2 = {'Data_recolha':post_data['Tempo_recolha'],
              'Categoria':post_data['Categoria'],'Empresa':post_data['Empresa'],
              'post_author_url':post_data['post_author_url'],
              #'Autor_Post_ID':post_data['Autor_Post_ID'], 
              'N_post':post_data['n'],
              'URL_Post': post_data['URL_Post'],'Texto_Post' : post_data['Texto_Post'], 'Data_Post' : post_data['Data_Post'] ,
              'Qtd_post_video':post_data['Tipo_post_video'], 'Qtd_post_foto':post_data['Tipo_post_foto'], 
              'Qtd_post_evento':post_data['Tipo_post_evento'], 'Qtd_post_link':post_data['Tipo_post_link'],
              'Reagiram_Likes': post_data['all'], 'Curtiram': post_data['1'], 'Amei': post_data['2'],'Uau': post_data['3'],
              'Haha': post_data['4'],'Triste': post_data['7'],'Grr': post_data['8'],
              'Partilhas': post_data['Partilhas'], #'A': post_data['5'],'B': post_data['6'],
              'Likes_Comentario' : post_data['Likes_Comentario'],
              'Autor': post_data['Autor'], 'Autor_id' : post_data['Autor_id'], 'Comentario': post_data['Comentario'],
              'Comentario_id': post_data['Comentario_id'], 'Data_Comentario': post_data['Data_Comentario'], 
              'Flag_Reply': post_data['Flag_Reply'],
              'Count_Replies': post_data['Replies']
             }

post_data3 = pd.DataFrame.from_dict(post_data2, orient='index')
base = post_data3.transpose()

sticker = base['Autor'] == base['Comentario']
base.loc[sticker,'Comentario'] = ''

fotos = base.loc[(base.Qtd_post_video==0) & (base.Qtd_post_foto==0) & (base.Qtd_post_evento==0) & (base.Qtd_post_link==0),'URL_Post'].str.contains(pat = 'photo')
base.loc[fotos.index,'Qtd_post_foto'] = 1

videos = base.loc[(base.Qtd_post_video==0) & (base.Qtd_post_foto==0) & (base.Qtd_post_evento==0) & (base.Qtd_post_link==0),'URL_Post'].str.contains(pat = 'video')
base.loc[videos.index,'Qtd_post_video'] = 1

base

,Data_recolha,Categoria,Empresa,post_author_url,N_post,URL_Post,Texto_Post,Data_Post,Qtd_post_video,Qtd_post_foto,...,Grr,Partilhas,Likes_Comentario,Autor,Autor_id,Comentario,Comentario_id,Data_Comentario,Flag_Reply,Count_Replies
0,2021-03-25 09:35:23.253915,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,1,https://mobile.facebook.com/story.php?story_fb...,Celebra-se hoje a data que simboliza o papel d...,22 h,0,1,...,0.0,13,NaN,NaN,NaN,,NaN,NaN,,
1,2021-03-25 09:35:31.009199,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,2,https://mobile.facebook.com/story.php?story_fb...,Atendendo às inúmeras dificuldades originadas ...,23 de março às 18:30,0,1,...,0.0,12,NaN,NaN,NaN,,NaN,NaN,,
2,2021-03-25 09:35:38.825124,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,3,https://mobile.facebook.com/story.php?story_fb...,"📣 CANDIDATURAS ABERTAS📌21, 22 e 23 Junho 2021V...",23 de março às 14:33,1,0,...,0.0,10,NaN,NaN,NaN,,NaN,NaN,,
3,2021-03-25 09:35:46.766351,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,4,https://mobile.facebook.com/story.php?story_fb...,💧2.2 biliões de pessoas vivem sem acesso a águ...,22 de março às 16:00,0,1,...,0.0,3,NaN,NaN,NaN,,NaN,NaN,,
4,2021-03-25 09:35:54.777808,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,5,https://mobile.facebook.com/story.php?story_fb...,📣 Pós-Graduação em Implantologia - Candidatura...,22 de março às 13:00,0,1,...,0.0,19,NaN,NaN,NaN,,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,NaT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,,0
4352,NaT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,,
4353,NaT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,,
4354,NaT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,,0


In [29]:
base1 = base
base1.Flag_Reply = pd.to_numeric(base1.Flag_Reply)
# base1.Count_Replies = pd.to_numeric(base1.Count_Replies)
base1 = base1.fillna(0)
#base1 = base1.sort_index(ascending=False)
#base1
base1["Qt_Replies"] = base1["Flag_Reply"]

data = base1.Qt_Replies

In [30]:
for i in range(0, len(data)-1):   
        j = 1            
        while data[i+j] == 1:  
            data[i] += data[i + j]
            j = j+1
## It's gonna show a red alert! That's good        

<ipython-input-30-51d8a45c4b32>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] += data[i + j]


In [31]:
base1

,Data_recolha,Categoria,Empresa,post_author_url,N_post,URL_Post,Texto_Post,Data_Post,Qtd_post_video,Qtd_post_foto,...,Partilhas,Likes_Comentario,Autor,Autor_id,Comentario,Comentario_id,Data_Comentario,Flag_Reply,Count_Replies,Qt_Replies
0,2021-03-25 09:35:23.253915,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,1.0,https://mobile.facebook.com/story.php?story_fb...,Celebra-se hoje a data que simboliza o papel d...,22 h,0.0,1.0,...,13,NaN,NaN,NaN,,NaN,NaN,0.0,,0.0
1,2021-03-25 09:35:31.009199,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,2.0,https://mobile.facebook.com/story.php?story_fb...,Atendendo às inúmeras dificuldades originadas ...,23 de março às 18:30,0.0,1.0,...,12,NaN,NaN,NaN,,NaN,NaN,0.0,,0.0
2,2021-03-25 09:35:38.825124,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,3.0,https://mobile.facebook.com/story.php?story_fb...,"📣 CANDIDATURAS ABERTAS📌21, 22 e 23 Junho 2021V...",23 de março às 14:33,1.0,0.0,...,10,NaN,NaN,NaN,,NaN,NaN,0.0,,0.0
3,2021-03-25 09:35:46.766351,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,4.0,https://mobile.facebook.com/story.php?story_fb...,💧2.2 biliões de pessoas vivem sem acesso a águ...,22 de março às 16:00,0.0,1.0,...,3,NaN,NaN,NaN,,NaN,NaN,0.0,,0.0
4,2021-03-25 09:35:54.777808,Academia,Egas Moniz Cooperativa de Ensino,https://www.facebook.com/uegas,5.0,https://mobile.facebook.com/story.php?story_fb...,📣 Pós-Graduação em Implantologia - Candidatura...,22 de março às 13:00,0.0,1.0,...,19,NaN,NaN,NaN,,NaN,NaN,0.0,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0.0,0,0.0
4352,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0.0,,0.0
4353,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0.0,,0.0
4354,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0.0,0,0.0


<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Let's burn the ring: Export the Dataset  </b>

Remember to save with the category name

In [32]:
# This one is right

writer = pd.ExcelWriter('C:/Users/GSantos/Documents/GABRIEL RAVI/Web Scrapping/WS_Facebook_2/Bases Prontas/FB_0_4.xlsx',
                        engine='xlsxwriter',options={'strings_to_urls': False}) #.format(staName2)
base1.to_excel(writer)
writer.close()

<hr>
<b style='color: #30475e;font-size:24px' id="data_understanding"> Excellent young warrior, now middle earth is safe again... not yet </b>